<h6> Ekrem Çetinkaya S004228 </h6>
<hr /> <hr />
<h1> <strong> Cifar-10 Trainer </strong> </h1>
<hr /> <hr />

Read data and do preprocessing

In [5]:
import read_data
import numpy as np
def normalize(images):
    maximum = np.max(images)
    minimum = np.min(images)
    
    return (images - minimum) / (maximum - minimum)

def one_hot_encode(labels):
    nx = np.max(labels) + 1
    
    return np.eye(nx)[labels]
read_data.preprocess_and_save_data(normalize, one_hot_encode)

<h3> CHECKPOINT </h3>
<br>
Run from here if you executed above cell before

In [1]:
import pickle
valid_images, valid_labels = pickle.load(open('preprocess_validation.p', mode='rb'))

Network Inputs

In [2]:
import tensorflow as tf
def image_input(image_shape):
    return tf.placeholder(tf.float32, [None, image_shape[0], image_shape[1], 3], name='image')

def label_input(class_count):
    return tf.placeholder(tf.float32, [None, class_count], name='label')

def dropout_input():
    return tf.placeholder(tf.float32, name='keep_prob')

tf.reset_default_graph()

<h4> <strong> First CNN </strong> </h4>
<ul>
<li> <strong> Convolution </strong> 3 x 3 filter, 64 features </li>
<li> <strong> Max Pool </strong> 3 x 3 size, stride 2 </li>
<li> <strong> Dropout </strong> 50% probability</li>
<li> <strong> Convolution </strong> 5 x 5 filter, 64 features </li>
<li> <strong> Max Pool </strong> 3 x 3 size, stride 2 </li>
<li> <strong> Dropout </strong> 50% probability</li>
<li> <strong> Convolution </strong> 3 x 3 filter, 64 features </li>
<li> <strong> Max Pool </strong> 2 x 2 size, stride 2 </li>
<li> <strong> Dropout </strong> 50% probability</li>
<li> <strong> Flatten layer </strong> </li>
<li> <strong> Fully connected layer </strong> 192 </li>
<li> <strong> Softmax layer </strong> </li>

In [ ]:
import model
def cnn_model_one(x_tensor):
    conv1 = model.conv2d(x_tensor, conv_features=64, conv_filter=[3,3], conv_strides=[1,1])
    max_pool1 = model.max_pool(conv1, pool_size=[3,3], pool_stride=[2,2])
    tf.nn.dropout(max_pool1, 0.5)
    
    conv2 = model.conv2d(max_pool1, 64, [5,5], [1,1])
    max_pool2 = model.max_pool(conv2, [3,3], [2,2])
    tf.nn.dropout(max_pool2, 0.5)
    
    conv3 = model.conv2d(max_pool2, 64, [3,3], [1,1])
    max_pool3 = model.max_pool(conv3, [2,2], [2,2])
    tf.nn.dropout(max_pool3, 0.5)
    
    flattened = model.flatten(max_pool3)
    fc1 = model.fully_conn(flattened, 192)
    
    return model.output(fc1, 10)

<h4> <strong> Second CNN </strong> </h4>
<ul>
<li> <strong> Convolution </strong> 3 x 3 filter, 32 features </li>
<li> <strong> Convolution </strong> 3 x 3 filter, 32 features </li>
<li> <strong> Max Pool </strong> 2 x 2 size, stride 2 </li>
<li> <strong> Dropout </strong> 25% probability</li>
<li> <strong> Convolution </strong> 3 x 3 filter, 64 features </li>
<li> <strong> Convolution </strong> 3 x 3 filter, 64 features </li>
<li> <strong> Max Pool </strong> 2 x 2 size, stride 2 </li>
<li> <strong> Dropout </strong> 25% probability</li>
<li> <strong> Flatten layer </strong> </li>
<li> <strong> Fully connected layer </strong> 128 </li>
<li> <strong> Dropout </strong> 50% probability</li>
<li> <strong> Softmax layer </strong> </li>

In [10]:
def cnn_model_two(x_tensor):
    #1M PARAMETER %52 5 BATCH 1 ITERATION
    conv1 = model.conv2d(x_tensor, conv_features=32, conv_filter=[3,3], conv_strides=[1,1])
    conv2 = model.conv2d(conv1, 32, [3,3], [1,1])
    max_pool1 = model.max_pool(conv2, pool_size=[2,2], pool_stride=[2,2])
    tf.nn.dropout(max_pool1, 0.25)
    
    conv3 = model.conv2d(max_pool1, 64, [3,3], [1,1])
    conv4 = model.conv2d(conv3, 64, [3,3], [1,1])
    max_pool2 = model.max_pool(conv4, [2,2], [2,2])
    tf.nn.dropout(max_pool2, 0.25)
    
    flattened = model.flatten(max_pool2)
    fc1 = model.fully_conn(flattened, 128)
    tf.nn.dropout(fc1, 0.5)
    
    return model.output(fc1, 10)

<h5> Define variables for training </h5>

In [ ]:
# Inputs
images = image_input((32, 32, 3))
labels = label_input(10)
# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(cost)
# Accuracy
correct_result = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_result, tf.float32), name='accuracy')

<h5> Select which model to train </h5>

In [ ]:
logits = cnn_model_one(images)
save_model_path =".\\cifar_model_one"
# logits = cnn_model_two(images)
# save_model_path = ".\\cifar_model_two"

logits = tf.identity(logits, name='logits')

<h5> Specify iteration count and batch size </h5>

In [13]:
MAX_ITERATIONS = 1
BATCH_SIZE = 128

<h5> Training Functions </h5>

In [ ]:
import read_data

def train_network(session ,optimizer, image_batch, label_batch):
    session.run(optimizer, feed_dict={images: image_batch, labels: label_batch})

def print_stats(session, feature_batch, label_batch, cost, accuracy):
    global valid_features, valid_labels
    validation_accuracy = session.run(
        accuracy,
        feed_dict={
            images: valid_images,
            labels: valid_labels,
        }
    )
    cost = session.run(
        cost,
        feed_dict={
            images: feature_batch,
            labels: label_batch,
        }
    )
    print('Cost = {0} - Validation Accuracy = {1}'.format(cost, validation_accuracy))

def train_with_all_batches():
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for iteration in range(MAX_ITERATIONS):
            num_of_batches = 5
            for batch_i in range(1, num_of_batches + 1):
                print('Iteration {:>2}, CIFAR-10 Batch {}:  '.format(iteration + 1, batch_i), end='')
                for batch_images, batch_labels in read_data.load_preprocess_training_batch(batch_i, BATCH_SIZE):
                    train_network(sess, optimizer, batch_images, batch_labels)
                    print_stats(sess, batch_images, batch_labels, cost, accuracy)  
    # Save Model
    saver = tf.train.Saver()
    final_save_path = saver.save(sess, save_model_path)

def train_with_one_batch():    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for iteration in range(MAX_ITERATIONS):
            batch_i = 1
            print('Iteration {:>2}, CIFAR-10 Batch {}:  '.format(iteration + 1, batch_i), end='')
            for batch_images, batch_labels in read_data.load_preprocess_training_batch(batch_i, BATCH_SIZE):
                train_network(sess, optimizer, batch_images, batch_labels)
                print_stats(sess, batch_images, batch_labels, cost, accuracy)
        
        saver = tf.train.Saver()
        final_save_path = saver.save(sess, save_model_path)

<h3> Train Network </h3>

In [ ]:
train_with_one_batch()
# train_with_all_batches()